In [ ]:
import pandas as pd
df = pd.read_csv('/content/customer_support_tickets.csv')
df.head()


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [ ]:
print(df.iloc[:4,9])

0    I'm having an issue with the {product_purchase...
1    I'm having an issue with the {product_purchase...
2    I'm facing a problem with my {product_purchase...
3    I'm having an issue with the {product_purchase...
Name: Ticket Description, dtype: object


In [ ]:
# Replace {product_purchased} with 'product' in the Ticket Description column
df['Ticket Description'] = df['Ticket Description'].str.replace(r'\{product_purchased\}', 'product', regex=True)

# Check the result
print(df['Ticket Description'].head())


0    I'm having an issue with the product. Please a...
1    I'm having an issue with the product. Please a...
2    I'm facing a problem with my product. The prod...
3    I'm having an issue with the product. Please a...
4    I'm having an issue with the product. Please a...
Name: Ticket Description, dtype: object


In [ ]:
df = df.drop_duplicates()

In [ ]:
df_columns = ['Ticket ID', 'Customer Name', 'Customer Email',
        'Product Purchased', 'Date of Purchase',
        'Ticket Subject', 'Ticket Status',
       'Resolution', 'Ticket Priority', 'Ticket Channel',
       'First Response Time', 'Time to Resolution',
       'Customer Satisfaction Rating']

In [ ]:
df = df.drop(columns = df_columns)

In [ ]:
df.head()

,Customer Age,Customer Gender,Ticket Type,Ticket Description
0,32,Other,Technical issue,I'm having an issue with the product. Please a...
1,42,Female,Technical issue,I'm having an issue with the product. Please a...
2,48,Other,Technical issue,I'm facing a problem with my product. The prod...
3,27,Female,Billing inquiry,I'm having an issue with the product. Please a...
4,67,Female,Billing inquiry,I'm having an issue with the product. Please a...


In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# Download required NLTK data files if not already downloaded
nltk.download('stopwords')
nltk.download('wordnet')

def clean_text(texts):
    """
    A robust function to clean a list of text documents that handles NaN values.
    """
    # Initialize tools
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    cleaned_texts = []
    for text in texts:
        # Check if text is NaN or None
        if pd.isna(text) or text is None:
            cleaned_texts.append("")  # Append empty string for NaN values
            continue

        # Convert to string in case it's not
        text = str(text)

        # 1. Lowercase
        text = text.lower()

        # 2. Remove punctuation and numbers
        text = re.sub(r'[^a-zA-Z\s]', '', text)

        # 3. Tokenize (split into words)
        words = text.split()

        # 4. Remove stopwords and lemmatize
        cleaned_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words and len(word) > 2]

        # 5. Re-join words into a single string
        cleaned_text = ' '.join(cleaned_words)
        cleaned_texts.append(cleaned_text)

    return cleaned_texts


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Assuming your DataFrame is named df
df['Cleaned_Ticket_Description'] = clean_text(df['Ticket Description'])


In [ ]:
df.head()

,Customer Age,Customer Gender,Ticket Type,Ticket Description,Cleaned_Ticket_Description
0,32,Other,Technical issue,I'm having an issue with the product. Please a...,issue product please assist billing zip code a...
1,42,Female,Technical issue,I'm having an issue with the product. Please a...,issue product please assist need change existi...
2,48,Other,Technical issue,I'm facing a problem with my product. The prod...,facing problem product product turning working...
3,27,Female,Billing inquiry,I'm having an issue with the product. Please a...,issue product please assist problem youre inte...
4,67,Female,Billing inquiry,I'm having an issue with the product. Please a...,issue product please assist note seller respon...


In [ ]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 5.9 MB/s eta 0:00:00


In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


In [ ]:
# Required imports
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

def fit_bertopic(texts, model_name="all-MiniLM-L6-v2", n_topics="auto"):
    """
    Fit a BERTopic model on a list of texts.

    Parameters:
    - texts: list of str
    - model_name: str, sentence-transformers model
    - n_topics: int or "auto", number of topics

    Returns:
    - topic_model: fitted BERTopic model
    - topics: list of topic assignments for each text
    - probs: list of probabilities for each topic assignment
    """
    # Initialize embedding model
    embedding_model = SentenceTransformer(model_name)

    # Initialize BERTopic model
    topic_model = BERTopic(embedding_model=embedding_model, nr_topics=n_topics, calculate_probabilities=True)

    # Fit the model and transform the texts
    topics, probs = topic_model.fit_transform(texts)

    return topic_model, topics, probs


In [ ]:
topic_model, topics, probs = fit_bertopic(df['Cleaned_Ticket_Description'])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1450,-1_product_please_issue_ive,"[product, please, issue, ive, assist, already,...",[seems glitch product software freeze frequent...
1,0,453,0_software_update_occurring_recent,"[software, update, occurring, recent, havent, ...",[issue product please assist problem started o...
2,1,389,1_followed_manual_mentioned_user,"[followed, manual, mentioned, user, troublesho...",[issue product please assist ive followed trou...
3,2,365,2_unexpectedly_act_intermittent_sometimes,"[unexpectedly, act, intermittent, sometimes, f...",[issue product please assist issue facing inte...
4,3,344,3_afterward_firmware_happening_related,"[afterward, firmware, happening, related, upda...",[issue product please assist ive recently upda...
5,4,325,4_performed_hoping_factory_reset,"[performed, hoping, factory, reset, didnt, hel...",[issue product please assist ive performed fac...
6,5,295,5_productivity_affecting_assistance_soon,"[productivity, affecting, assistance, soon, po...",[issue product please assist need assistance s...
7,6,257,6_peculiar_popping_errormessage_mean,"[peculiar, popping, errormessage, mean, messag...",[issue product please assist ive noticed pecul...
8,7,246,7_desired_find_guide_perform,"[desired, find, guide, perform, action, option...",[issue product please assist unable find optio...
9,8,224,8_battery_life_sudden_decrease,"[battery, life, sudden, decrease, longer, much...",[issue product please assist ive noticed sudde...


In [ ]:
probs.shape

(8469, 46)

In [ ]:
# Get topic names
topic_info = topic_model.get_topic_info()
topic_names = topic_info['Name'].tolist()

# Convert probabilities to DataFrame with topic names as columns
probs_df = pd.DataFrame(probs, columns=topic_names[:probs.shape[1]])

# Add to your original dataset
new_df = pd.concat([df, probs_df], axis=1)

In [ ]:
new_df.head()

,Customer Age,Customer Gender,Ticket Type,Ticket Description,Cleaned_Ticket_Description,-1_product_please_issue_ive,0_software_update_occurring_recent,1_followed_manual_mentioned_user,2_unexpectedly_act_intermittent_sometimes,3_afterward_firmware_happening_related,...,35_turning_yesterday_respond_working,36_heavily_hindering_rely_daily,37_affecting_assistance_soon_possible,38_lost_crashed_stored_data,39_turning_yesterday_respond_doesnt,40_deleted_peripheral_cable_adapter,41_manual_mentioned_user_payment,42_disqus_powered_javascript_enable,43_flickering_hardware_screen_concerned,44_turning_yesterday_respond_doesnt
0,32,Other,Technical issue,I'm having an issue with the product. Please a...,issue product please assist billing zip code a...,5.254693e-307,7.255234e-306,2.338142e-308,1.863174e-307,6.871059e-308,...,2.599063e-308,4.903499e-308,3.848607e-308,4.482610e-308,3.654569e-308,1.000000,5.821681e-308,3.909510e-308,3.862117e-308,7.660356e-308
1,42,Female,Technical issue,I'm having an issue with the product. Please a...,issue product please assist need change existi...,1.588265e-02,8.379455e-03,7.757630e-01,8.759261e-03,1.598986e-03,...,1.796488e-03,3.566698e-03,1.296848e-03,1.958435e-03,1.291633e-03,0.001676,1.613803e-03,1.505407e-03,1.196242e-03,1.724766e-03
2,48,Other,Technical issue,I'm facing a problem with my product. The prod...,facing problem product product turning working...,6.551582e-03,3.934891e-03,5.739662e-04,3.471281e-03,8.504878e-04,...,1.014579e-03,1.918208e-03,6.799988e-04,1.458408e-03,6.774172e-04,0.000804,9.287348e-04,8.778673e-04,1.018200e-03,1.003351e-03
3,27,Female,Billing inquiry,I'm having an issue with the product. Please a...,issue product please assist problem youre inte...,3.117996e-03,2.320563e-03,1.974608e-04,1.066744e-03,5.593374e-04,...,1.722992e-04,3.314585e-04,2.208781e-04,2.407166e-04,2.151250e-04,0.000453,3.354962e-04,3.607834e-04,2.331587e-04,2.917571e-04
4,67,Female,Billing inquiry,I'm having an issue with the product. Please a...,issue product please assist note seller respon...,1.168687e-02,6.975531e-03,8.189918e-04,4.888704e-03,1.457933e-03,...,1.291006e-03,2.450463e-03,1.318119e-03,1.648082e-03,1.307278e-03,0.001416,1.405945e-03,1.439498e-03,2.369009e-03,1.593458e-03


In [ ]:
y = new_df['Ticket Type']
x = new_df.drop(columns=['Ticket Type', 'Ticket Description', 'Cleaned_Ticket_Description'])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=4002)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
categorical_features = ['Customer Gender']
numeric_features = ['Customer Age',
        '-1_product_please_issue_ive',
       '0_software_update_occurring_recent',
       '1_followed_manual_mentioned_user',
       '2_unexpectedly_act_intermittent_sometimes',
       '3_afterward_firmware_happening_related',
       '4_performed_hoping_factory_reset',
       '5_productivity_affecting_assistance_soon',
       '6_peculiar_popping_errormessage_mean', '7_desired_find_guide_perform',
       '8_battery_life_sudden_decrease', '9_safe_concerned_security_ensure',
       '10_unresolved_remains_contacted_customer',
       '11_access_regain_displaying_credential',
       '12_charger_charging_came_original',
       '13_peripheral_cable_adapter_different',
       '14_consistently_occurs_feature_application',
       '15_might_hardwarerelated_worried_require',
       '16_configuration_setting_different_tried',
       '17_cache_clearing_app_tried', '18_unexpected_causing_app_loss',
       '19_internet_connection_establish_stable',
       '20_configured_correctly_everything_setting',
       '21_official_reviewed_website_troubleshooting',
       '22_flickering_hardware_screen_seems',
       '23_encountering_whenever_crash_fix',
       '24_password_account_forgotten_recover', '25_lost_stored_crashed_data',
       '26_connect_set_fails_take', '27_connecting_network_wifi_although',
       '28_noise_strange_functioning_suspect',
       '29_encountered_document_retrieve_disappeared',
       '30_locked_unlock_account_logging', '31_others_similar_reported_sure',
       '32_widespread_model_experiencing_seems',
       '33_glitch_freeze_frequently_difficult',
       '34_deleted_accidentally_important_urgently',
       '35_turning_yesterday_respond_working',
       '36_heavily_hindering_rely_daily',
       '37_affecting_assistance_soon_possible', '38_lost_crashed_stored_data',
       '39_turning_yesterday_respond_doesnt',
       '40_deleted_peripheral_cable_adapter',
       '41_manual_mentioned_user_payment',
       '42_disqus_powered_javascript_enable',
       '43_flickering_hardware_screen_concerned',
       '44_turning_yesterday_respond_doesnt']
# Create transformers for each type
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numeric_transformer = StandardScaler()
# Combine them using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numeric_features)
    ]
)
# Fit the preprocessor on training data only
preprocessor.fit(x_train)
x_train_processed = preprocessor.transform(x_train)
x_test_processed = preprocessor.transform(x_test)


In [ ]:
# Get feature names
encoded_cat_cols = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
processed_feature_names = list(encoded_cat_cols) + numeric_features
x_train_final = pd.DataFrame(x_train_processed.toarray() if hasattr(x_train_processed, "toarray") else x_train_processed,
                             columns=processed_feature_names, index=x_train.index)
x_test_final = pd.DataFrame(x_test_processed.toarray() if hasattr(x_test_processed, "toarray") else x_test_processed,
                            columns=processed_feature_names, index=x_test.index)

In [ ]:
# Import libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 200],
    'max_depth': [20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [4, 6, 10]
}
# Create the grid search object
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,                     # 5-fold cross-validation
    scoring='accuracy',       # could also try 'f1', 'precision', or 'recall'
    n_jobs=-1,                # use all cores
    verbose=1                 # show progress
)
# Fit the grid search on your training data
grid_search.fit(x_train_final, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [20, None],
                         'min_samples_leaf': [4, 6, 10],
                         'min_samples_split': [2, 5],
                         'n_estimators': [50, 200]},
             scoring='accuracy', verbose=1)

In [ ]:
# Get the best model
best_rf_model = grid_search.best_estimator_
y_train_pred = best_rf_model.predict(x_train_final)
print(classification_report(y_train, y_train_pred))

                      precision    recall  f1-score   support

     Billing inquiry       0.71      0.70      0.70      1164
Cancellation request       0.71      0.71      0.71      1177
     Product inquiry       0.75      0.67      0.71      1154
      Refund request       0.67      0.76      0.71      1201
     Technical issue       0.71      0.71      0.71      1232

            accuracy                           0.71      5928
           macro avg       0.71      0.71      0.71      5928
        weighted avg       0.71      0.71      0.71      5928



In [ ]:
y_test_pred = best_rf_model.predict(x_test_final)
print(classification_report(y_test, y_test_pred))

                      precision    recall  f1-score   support

     Billing inquiry       0.18      0.19      0.18       470
Cancellation request       0.20      0.20      0.20       518
     Product inquiry       0.18      0.15      0.16       487
      Refund request       0.23      0.23      0.23       551
     Technical issue       0.21      0.23      0.22       515

            accuracy                           0.20      2541
           macro avg       0.20      0.20      0.20      2541
        weighted avg       0.20      0.20      0.20      2541



In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_final = le.fit_transform(y_train)
y_test_final = le.fit_transform(y_test)

In [ ]:

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns


model = XGBClassifier(
    objective='multi:softprob',  # multi-class classification
    num_class=5,                 # number of target classes
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'       # multi-class log loss
)

# Fit the model
model.fit(x_train_final, y_train_final)


#  Predictions

y_pred = model.predict(x_test_final)

# Evaluation Metrics



print("Classification Report:\n")
print(classification_report(y_test_final, y_pred))




Classification Report:

              precision    recall  f1-score   support

           0       0.17      0.18      0.17       470
           1       0.22      0.21      0.21       518
           2       0.19      0.19      0.19       487
           3       0.20      0.19      0.19       551
           4       0.21      0.23      0.22       515

    accuracy                           0.20      2541
   macro avg       0.20      0.20      0.20      2541
weighted avg       0.20      0.20      0.20      2541

